In [1]:
import pandas as pd
import numpy as np
import math
from itertools import chain

In [2]:
df = pd.read_csv('/Users/Maryam/Documents/summer-Research-2020/ImgClassAnalysis/imgClassificationData.csv')


In [3]:
# Temporary way of pulling out the data we actually want to analyze. Confirm with Cindy first. 
# check if postQs are answered
# where/when to drop the nan values
data = df.dropna(subset=['Q75']).drop([0,1, 117, 123]).reset_index()
# data = df.iloc[[124]] //this worked for cindy
# data.dropna(subset=['Q75'])
# data = data[['Q75']].reset_index()
data

#  want to separate the data for inv and orig for each subject

,index,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,allStims11,allStims12,allStims13,allStims14,allStims15,allStims16,allStims17,allStims18,allStims19,allStims20
0,124,2021-03-04 17:22:16,2021-03-04 23:43:04,IP Address,169.229.122.114,100,22848,True,2021-03-04 23:43:05,R_Ww9Xrpbwjg6ubBv,...,"0438_inv,0438_ori,0444_ori,0444_inv,0435_inv,0...","0540_ori,0540_inv,0525_ori,0525_inv,0531_inv,0...","0565_ori,0565_inv,0588_inv,0588_ori,0578_inv,0...","0609_inv,0609_ori,0646_ori,0646_inv,0620_ori,0...","0688_inv,0688_ori,0658_ori,0658_inv,0692_inv,0...","0710_inv,0710_ori,0749_ori,0749_inv,0721_ori,0...","0765_ori,0765_inv,0783_inv,0783_ori,0775_ori,0...","0899_inv,0899_ori,0867_inv,0867_ori,0876_inv,0...","0947_ori,0947_inv,0903_inv,0903_ori,0941_ori,0...","0812_inv,0812_ori,0809_inv,0809_ori,0810_inv,0..."


In [5]:
# traversing through each row to pull out each subjects data
for index,subject_data in data.iterrows():
    subject_data = subject_data.to_frame().T
    # need to aggregate allStims and allSelections for each subject
    allStimuli = subject_data['allStims1']
    allSelects = subject_data['selections1']
    for i in range(2,21):
        stimCol = subject_data['allStims' + str(i)]
        allStimuli = pd.concat([stimCol,allStimuli])
        

        selectsCol = subject_data['selections' + str(i)]
        allSelects = pd.concat([selectsCol,allSelects])
    
# selecting every third entry for response column
    allStimuli = allStimuli.tolist()
    allSelects = allSelects.tolist()
    for i in range(len(allStimuli)):
        allStimuli[i] = allStimuli[i].split(',')
        allSelects[i] = allSelects[i].split(',')
    allStimuli = list(chain.from_iterable(allStimuli))
    allSelects = list(chain.from_iterable(allSelects))
    allSelects = allSelects[1:][::3]
        
# trying to figure out how to select inv and orig based on 37 and 39 keycode
    image = []
    stim = allStimuli[::2]
    stimuluses = []
    for i in range(len(allSelects)):
        if allSelects[i] == '37':
            if (stim[i].find('inv') != -1):
                image.append(-1)

            else:
                image.append(1)
            
            word = stim[i].split('_')
            stimuluses.append(word[0])
        elif allSelects[i] == '39':
            if (stim[i].find('inv') != -1):
                image.append(-1)
            else:
                image.append(1)
            
            word = stim[i].split('_')
            stimuluses.append(word[0])


# creating new desired tables for each subject
    # creating a dictionary to store all dataframes of data for each subject
    frames = {}
    
    # -1 or 1 if inv or orig, left=37 right=39
    desired = pd.DataFrame(columns=['stimulus', 'response', 'subject'], index=[i for i in range(0,1000)])
    desired['stimulus'] = stimuluses
    desired['response'] = image
    desired['subject'] = index + 1
    
    desired = desired.sort_values(by='stimulus').reset_index(drop=True)
    
    frames.update({index+1: desired}) #Appending to the dictionary

# len(allSelects), len(allStimuli[::2]), len(image)

frames[1] #returning the dictionary with all the frames, you can index through them to get each subject's data

,stimulus,response,subject
0,0001,1,1
1,0002,-1,1
2,0003,-1,1
3,0004,1,1
4,0005,1,1
...,...,...,...
995,0996,-1,1
996,0997,-1,1
997,0998,1,1
998,0999,1,1
